# Fetch Reward DA Exercise
Question #1 : converting unstructure data into structure one (json.gz file > json file > csv file)
<br>
Author: Chengwu Weng
<br>
Date: 8/26/2021

In [1]:
import gzip
import json
import csv
import pandas as pd
import time
from datetime import datetime
pd.set_option('display.max_columns', None)

# Data transformation

## Json.gz to json

In [3]:
# open `brands.json.gz` and convert to `brands.json`
brands = []
for line in gzip.open(filename = 'brands.json.gz', mode='r'):
    brands.append(json.loads(line))
    
with open('brands.json', 'w') as f:
    json.dump(brands, f)

In [4]:
def transform_brands(brands):
    for i in brands:
        i['cpg_id'] = i['cpg']['$id']['$oid'] if 'cpg' in i and '$id' in i['cpg'] and '$oid' in i['cpg']['$id'] else None
        i['cpg_ref'] = i['cpg']['$ref'] if 'cpg' in i and '$ref' in i['cpg'] else None
    return brands

In [5]:
brands = transform_brands(brands)

In [6]:
len(brands)

1167

In [7]:
# open `receipts.json.gz` and convert to `receipts.json`
receipts = []
for line in gzip.open(filename = 'receipts.json.gz', mode='r'):
    receipts.append(json.loads(line))

with open('receipts.json', 'w') as f:
    json.dump(receipts, f)

In [8]:
len(receipts)

1119

In [9]:
# open `users.json.gz` and convert to `users.json`
users = []
for line in gzip.open(filename = 'users.json.gz', mode='r'):
    users.append(json.loads(line))

with open('users.json', 'w') as f:
    json.dump(users, f)

In [10]:
len(users)

495

## Json to csv

In [11]:
# write to csv from list of dictionary
def list_to_csv(data = brands, name = 'brands'):
    toCSV = data
    filename = "%s.csv" % name
    keys = set().union(*(d.keys() for d in toCSV))
    with open(filename, 'w', encoding='utf8', newline='') as output_file:
        fc = csv.DictWriter(output_file, 
                            fieldnames = keys) # toCSV[3].keys()
        fc.writeheader()
        fc.writerows(toCSV)

In [12]:
def get_rewardreceiptitemlist(receipts):
    rewardsReceiptItemList = []
    for i in receipts:
        if 'rewardsReceiptItemList' in i:
            for j in i['rewardsReceiptItemList']:
                j['receiptid'] = i['_id']['$oid']
                rewardsReceiptItemList.append(j)
    return rewardsReceiptItemList

In [13]:
rewardsReceiptItemList = get_rewardreceiptitemlist(receipts)

list_to_csv(brands, 'brands')
list_to_csv(receipts, 'receipts')
list_to_csv(users, 'users')
list_to_csv(rewardsReceiptItemList, 'rewardsReceiptItemList')

## Date

In [14]:
data_brands = pd.DataFrame.from_records(brands)
data_receipts = pd.DataFrame.from_records(receipts)
data_users = pd.DataFrame.from_records(users)
data_rewardsReceiptItemList = pd.DataFrame.from_records(rewardsReceiptItemList)
# https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe/33020669#33020669

In [15]:
data_brands['_id'] = data_brands['_id'].apply(lambda x: x.get('$oid'))
data_receipts['_id'] = data_receipts['_id'].apply(lambda x: x.get('$oid'))
data_users['_id'] = data_users['_id'].apply(lambda x: x.get('$oid'))

In [16]:
def get_date(df, col):
    for i in range(len(df)):
        if ~pd.isna(df[col][i]) == -1 : 
            df[col][i] = df[col][i].get('$date')
        else: continue

In [17]:
get_date(data_receipts, 'createDate')
get_date(data_receipts, 'dateScanned')
get_date(data_receipts, 'finishedDate')
get_date(data_receipts, 'pointsAwardedDate')
get_date(data_receipts, 'purchaseDate')
get_date(data_receipts, 'modifyDate')
get_date(data_users, 'createdDate')
get_date(data_users, 'lastLogin')

E:\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [18]:
# https://blog.csdn.net/woddle/article/details/97389197?utm_medium=distribute.pc_relevant_t0.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-1.control&depth_1-utm_source=distribute.pc_relevant_t0.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-1.control
def timestamp_to_datetime(timestamp):
    local_dt_time = datetime.fromtimestamp(timestamp / 1000.0)
    return local_dt_time

def df_to_datetime(df, col):
    for i in range(len(df)):
        if ~pd.isna(df[col][i]) == -1 : 
            df[col][i] = timestamp_to_datetime(df[col][i])
        else: continue

In [19]:
df_to_datetime(data_receipts, 'createDate')
df_to_datetime(data_receipts, 'dateScanned')
df_to_datetime(data_receipts, 'finishedDate')
df_to_datetime(data_receipts, 'pointsAwardedDate')
df_to_datetime(data_receipts, 'purchaseDate')
df_to_datetime(data_receipts, 'modifyDate')
df_to_datetime(data_users, 'createdDate')
df_to_datetime(data_users, 'lastLogin')

E:\anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [20]:
data_brands.to_csv(r'brands.csv',index=False)
data_receipts.to_csv(r'receipts.csv',index=False)
data_users.to_csv(r'users.csv',index=False)